In [1]:
import nltk
import gensim
import numpy as np
import pandas as pd
import re,string,unicodedata

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.porter import PorterStemmer
from gensim.utils import simple_preprocess 
from gensim.models import Phrases, Word2Vec
from sklearn.preprocessing import LabelBinarizer
from nltk.tokenize import word_tokenize,sent_tokenize


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
# Load the IMDb dataset from the CSV file
imdb_df = pd.read_csv('/content/irony-labeled.csv', header=None, names=['comment_text', 'label'])
imdb_df["label"] = np.where(imdb_df["label"] == "-1", 1, 0)

gen_df = pd.read_csv('/GEN-sarc-notsarc.csv', header=None, names=['class', 'text'])
gen_df = gen_df.append(pd.read_csv('/HYP-sarc-notsarc.csv'))
gen_df = gen_df.append(pd.read_csv('/RQ-sarc-notsarc.csv'))

gen_df["class"] = np.where(gen_df["class"] == "notsarc", 0, 1)

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

# Preprocess the dataset
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

<ipython-input-29-a42077d2f61c>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen_df = gen_df.append(pd.read_csv('/HYP-sarc-notsarc.csv'))
<ipython-input-29-a42077d2f61c>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gen_df = gen_df.append(pd.read_csv('/RQ-sarc-notsarc.csv'))


In [30]:
imdb_df['comment_text']=imdb_df['comment_text'].apply(denoise_text)
gen_df['text']=gen_df['text'].apply(denoise_text)

<ipython-input-29-a42077d2f61c>:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [31]:
words, imdb_words, gen_words = [], [], []
for i in imdb_df.comment_text.values:
    words.append(i.split())
    imdb_words.append(i.split())
words=words[1:]

for i in gen_df.text.values:
    words.append(i.split())
    gen_words.append(i.split())

In [32]:
len(words)

11336

In [33]:
imdb_wv_model = Word2Vec(words, vector_size=100, window=3, min_count=1, workers=16)

In [34]:
print(imdb_wv_model)

Word2Vec<vocab=53846, vector_size=100, alpha=0.025>


In [35]:
print(len(words))

11336


In [36]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(53846)
tokenizer.fit_on_texts(words)
text = tokenizer.texts_to_sequences(words)
text = keras.utils.pad_sequences(text, 50)

print(text[:2])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0   840   610  7758  5688   668   192     3  5689    86
    359  1035  4569 19649 19650 19651   100  2416  5058  1239  4570 19652
    289     8  5690   777   206   668 12542   381  1688   175  2000   175
    329   516]
 [  528   424 19653   134    26   757  1317  5059   841   117  2292   517
    107  1450 19654  3540   669   311 19655  2293   710  1171   257  9557
   2894     8 19656 19657     8  7759 12543 19658 19659  2293   710  2894
   1567   378    64    76  5060    17  4571  9558    92    27  4158   469
   6613  4571]]


In [37]:
y = imdb_df["label"]

In [38]:
all(y)

False

In [39]:
txt = tokenizer.texts_to_sequences(imdb_words)
txt = keras.utils.pad_sequences(txt, 50)

txt[:2]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,   840,   610,  7758,
         5688,   668,   192,     3,  5689,    86,   359,  1035,  4569,
        19649, 19650, 19651,   100,  2416,  5058,  1239,  4570, 19652,
          289,     8,  5690,   777,   206,   668, 12542,   381,  1688,
          175,  2000,   175,   329,   516]], dtype=int32)

In [40]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(np.array(txt), np.array(y), train_size=0.8, stratify=y)

In [41]:
import tensorflow as tf

model = keras.models.Sequential([

    # Embedding layer
    keras.layers.Embedding(input_dim=len(imdb_wv_model.wv.key_to_index),
                            output_dim=100,
                            input_length=50,
                            weights=[imdb_wv_model.wv.vectors],
                            trainable=True),
    
    keras.layers.Dropout(0.2),

    # BiLSTM layer
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),

    keras.layers.Bidirectional(keras.layers.LSTM(64)),

    keras.layers.Dropout(0.2),

    keras.layers.Dense(200),

    keras.layers.Activation('relu'),

    keras.layers.Dropout(0.2),
    
    # Output layer
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 100)           5384600   
                                                                 
 dropout (Dropout)           (None, 50, 100)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 50, 256)          234496    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 200)               2

In [ ]:
val_recall = 0.8160
val_precision = 0.8572
val_f1 = (2 * val_precision * val_recall) / (val_precision + val_recall)

val_f1

0.8360927563949319

In [42]:
txt2 = tokenizer.texts_to_sequences(gen_words)
txt2 = keras.utils.pad_sequences(txt2, 50)

y = gen_df["class"]

x_train, x_test, y_train, y_test = train_test_split(np.array(txt2), np.array(y), train_size=0.8, stratify=y)

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

Epoch 1/10
235/235 [==============================] - 37s 158ms/step - loss: 0.4500 - accuracy: 0.8465 - precision: 0.8474 - recall: 0.9986 - val_loss: 0.4125 - val_accuracy: 0.8472 - val_precision: 0.8472 - val_recall: 1.0000
Epoch 2/10
235/235 [==============================] - 17s 73ms/step - loss: 0.3273 - accuracy: 0.8776 - precision: 0.8860 - recall: 0.9819 - val_loss: 0.5256 - val_accuracy: 0.7971 - val_precision: 0.8513 - val_recall: 0.9214
Epoch 3/10
235/235 [==============================] - 10s 44ms/step - loss: 0.1039 - accuracy: 0.9636 - precision: 0.9749 - recall: 0.9824 - val_loss: 0.8691 - val_accuracy: 0.6917 - val_precision: 0.8519 - val_recall: 0.7700
Epoch 4/10
235/235 [==============================] - 10s 42ms/step - loss: 0.0441 - accuracy: 0.9850 - precision: 0.9920 - recall: 0.9903 - val_loss: 0.9550 - val_accuracy: 0.6581 - val_precision: 0.8549 - val_recall: 0.7184
Epoch 5/10
235/235 [==============================] - 6s 25ms/step - loss: 0.0309 - accuracy: 0

In [43]:
val_recall = 0.8511 *100
val_precision = 0.8441 *100
val_f1 = (2 * val_precision * val_recall) / (val_precision + val_recall)

val_f1

84.75855474280321

In [44]:
model.save("/bilstm-with-intermediate-task-irony.h")

In [45]:
gen_wv_model = Word2Vec(gen_words, vector_size=100, window=3, min_count=1, workers=16)

In [46]:
print(gen_wv_model)

Word2Vec<vocab=46908, vector_size=100, alpha=0.025>


In [48]:
# tokenizer = Tokenizer(46908)
# tokenizer.fit_on_texts(gen_words)
txt2 = tokenizer.texts_to_sequences(gen_words)
txt2 = keras.utils.pad_sequences(txt2, 50)

model = keras.models.Sequential([

    # Embedding layer
    keras.layers.Embedding(input_dim=len(imdb_wv_model.wv.key_to_index),
                            output_dim=100,
                            input_length=50,
                            weights=[imdb_wv_model.wv.vectors],
                            trainable=True),
    
    keras.layers.Dropout(0.2),

    # BiLSTM layer
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),

    keras.layers.Bidirectional(keras.layers.LSTM(64)),

    keras.layers.Dropout(0.2),

    keras.layers.Dense(200),

    keras.layers.Activation('relu'),

    keras.layers.Dropout(0.2),
    
    # Output layer
    keras.layers.Dense(1, activation='sigmoid')
])


y = gen_df["class"]

x_train, x_test, y_train, y_test = train_test_split(np.array(txt2), np.array(y), train_size=0.8, stratify=y, random_state=42)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
118/118 [==============================] - 34s 212ms/step - loss: 0.4230 - accuracy: 0.8474 - precision_1: 0.8474 - recall_1: 1.0000 - val_loss: 0.4391 - val_accuracy: 0.8472 - val_precision_1: 0.8472 - val_recall_1: 1.0000
Epoch 2/10
118/118 [==============================] - 17s 148ms/step - loss: 0.4149 - accuracy: 0.8474 - precision_1: 0.8474 - recall_1: 1.0000 - val_loss: 0.4131 - val_accuracy: 0.8472 - val_precision_1: 0.8472 - val_recall_1: 1.0000
Epoch 3/10
118/118 [==============================] - 10s 85ms/step - loss: 0.2181 - accuracy: 0.9134 - precision_1: 0.9316 - recall_1: 0.9690 - val_loss: 0.5392 - val_accuracy: 0.8099 - val_precision_1: 0.8579 - val_recall_1: 0.9296
Epoch 4/10
118/118 [==============================] - 10s 84ms/step - loss: 0.0634 - accuracy: 0.9806 - precision_1: 0.9895 - recall_1: 0.9876 - val_loss: 0.6430 - val_accuracy: 0.7726 - val_precision_1: 0.8642 - val_recall_1: 0.8680
Epoch 5/10
118/118 [==============================] - 8s 67ms/

In [49]:
val_recall = 0.8576 *100
val_precision = 0.8102 *100
val_f1 = (2 * val_precision * val_recall) / (val_precision + val_recall)

val_f1

83.32264300275813

In [ ]:
!zip -r "/bilstm.zip" "/bert-with-intermediate-task.h"

  adding: bert-with-intermediate-task.h/ (stored 0%)
  adding: bert-with-intermediate-task.h/assets/ (stored 0%)
  adding: bert-with-intermediate-task.h/keras_metadata.pb (deflated 92%)
  adding: bert-with-intermediate-task.h/variables/ (stored 0%)
  adding: bert-with-intermediate-task.h/variables/variables.index (deflated 67%)
  adding: bert-with-intermediate-task.h/variables/variables.data-00000-of-00001 (deflated 48%)
  adding: bert-with-intermediate-task.h/fingerprint.pb (stored 0%)
  adding: bert-with-intermediate-task.h/saved_model.pb (deflated 90%)
